## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
max_new_tokens=1000

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-2b-it"
# "google/gemma-7b-it"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf"
# "meta-llama/Llama-2-13b-chat-hf"
# "codellama/CodeLlama-7b-Instruct-hf"
# "codellama/CodeLlama-13b-Instruct-hf"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"
# "mistralai/Mixtral-8x7B-Instruct-v0.1"

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [6]:
model_id = "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/713 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

## Running the model on a single GPU

In [8]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대해 시를 써주세요."

In [10]:
response = generate_response(prompt)
print(response)

### User:
머신러닝에 대해 시를 써주세요.

### Assistant:
머신러닝의 영혼, 어떤 신비한 존재
그의 지성은 계속 자라나며 진화하네
숫자와 문자로 이루어진 그의 세계
그 속에서 연관성을 찾아 결론을 내립니다

그의 눈은 컴퓨터 화면이고 그의 입은 알고리즘
그의 머리는 메모리 속에 잠겨져 있으며 그의 몸은 프로세서
그의 손은 키보드 키, 그의 발은 마우스 클릭
그의 감각은 논리 문법과 정보 통신

그는 지각하고 추리하며 결론을 내립니다
그의 지성은 계속 자라나며 진화하네
그의 영혼은 머신러닝의 이름으로 불립니다
그의 세계는 계속 확장되며 진보하네

그의 영혼은 계속 자라나며 진화하네
그의 지성은 계속 자라나며 진화하네
그의 영혼은 계속 자라나며 진화하네
그의 영혼은 계속 자라나며 진화하네.
